In [ ]:
#pake EntsoeRawClient. Bagus kalo mau pamer skill ngequery raw data, tapi developnya relatif lama krn harus built parser tipe generasi, parser resolution di timeseries jadi akumulasi yg akurat, dsb

import os 
from dotenv import load_dotenv
import requests
from google.oauth2 import service_account
import xmltodict
import json
import pandas as pd 
from requests import request
import pytz
from bs4 import BeautifulSoup
from typing import Dict
from google.oauth2 import service_account
from google.cloud import storage

import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import os
import sys
import re

from pyspark.sql.types import StructType, StructField, ArrayType, FloatType, BooleanType
from pyspark.sql.types import DoubleType, IntegerType, StringType, DataType
from pyspark.sql import functions as F

from entsoe import EntsoeRawClient
from entsoe import EntsoePandasClient




In [13]:
#load env variables
load_dotenv('./creds/.env', verbose=True, override=True)

True

In [4]:
print(os.environ.get("PYSPARK_DRIVER_PYTHON"))

None


In [14]:


'''
----------------
INIT VARIABLES
----------------
'''

#setting up entsoe variables
security_token = os.environ.get("SECURITY_TOKEN")
ENTSOE_URL = 'https://transparency.entsoe.eu/api'

#setting up GCP variables
service_account_file = os.environ.get("SERVICE_ACCOUNT_FILE")
credentials = service_account.Credentials.from_service_account_file(
    service_account_file
)
gcs_bucket = os.environ.get("CLOUD_STORAGE_BUCKET")
print(gcs_bucket)

entsoe_analytics_1009


In [6]:



# #setting up session
# entsoe_session = requests.Session()

# '''
# ----------------
# SETTING UP FUNCTION CALLS 
# ----------------
# '''

# #format dates
# def datetime_to_str(dtm: pd.Timestamp) -> str:
#     #convert timezone to UTC if it's exist and it's not on UTC already
#     if dtm.tzinfo is not None and dtm.tzinfo != pytz.UTC:
#         dtm = dtm.tz_convert("UTC")
#     fmt = '%Y%m%d%H%M'
#     dtm_str = dtm.strftime(fmt)
#     return dtm_str

# #basic requests
# def basic_requests(params: Dict, start:pd.Timestamp, end: pd.Timestamp) -> requests.Response:
#     #setting up time intervals start and stop
#     start_str = datetime_to_str(start)
#     end_str = datetime_to_str(end)

#     #setting up params and extending with custom parameter based
#     base_params = {
#         'securityToken': security_token,
#         'periodStart': start_str,
#         'periodEnd': end_str,
#     }
#     params.update(base_params)
    
#     #seting up sesssion
#     session = requests.Session()
#     response = session.get(url=ENTSOE_URL, params=params)

#     return response

# # upload data to GCS
# def upload_blob_to_gcs(bucket_name, contents, destination_blob_name):
#     # Upload file to bucket"""

#     # ID of GCS bucket
#     # bucket_name =

#     # the contents from memory to be uploaded to file
#     # contents =

#     # the ID of your GCS object
#     # destination_blob_name =

#     storage_client = storage.Client(credentials=credentials)
#     bucket = storage_client.bucket(bucket_name)
#     blob = bucket.blob(destination_blob_name)

#     blob.upload_from_string(contents)


# '''
# ----------------
# EXTRACTION
# ----------------
# '''
# #for test, we'll be querying Actual Generation Output per Generation Unit

# domain = '10YCZ-CEPS-----N'
# params_requests = {
#     'documentType': 'A73',
#     'processType': 'A16',
#     'in_Domain': {domain},
# }
# start=pd.Timestamp('202101011200', tz='Europe/Berlin')
# end=pd.Timestamp('202101011300', tz='Europe/Berlin')

# entsoe_data = basic_requests(params=params_requests, start=start, end=end)
# # entsoe_dict = xmltodict.parse(entsoe_data.text)
# # entsoe_json = json.dumps(entsoe_dict, indent=4)
# # print(entsoe_json)
# print(entsoe_data.text)

# '''
# ----------------
# LOAD
# ----------------
# '''
# # #upload to GCS
# # landing_filename=f"entsoe_data_{start}.json"
# # upload_blob_to_gcs(bucket_name=gcs_bucket, contents=entsoe_json, destination_blob_name=landing_filename)

# #upload to GCS - XML
# landing_filename=f"entsoe_data_{start}_{domain}.xml"
# upload_blob_to_gcs(bucket_name=gcs_bucket, contents=entsoe_data.text, destination_blob_name=landing_filename)



In [19]:

#setting up session
entsoe_client = EntsoeRawClient(security_token)
'''
----------------
SETTING UP FUNCTION CALLS 
----------------
'''

# upload data to GCS
def upload_blob_to_gcs(bucket_name, contents, destination_blob_name):
    # Upload file to bucket"""

    # ID of GCS bucket
    # bucket_name =

    # the contents from memory to be uploaded to file
    # contents =

    # the ID of your GCS object
    # destination_blob_name =

    storage_client = storage.Client(credentials=credentials)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_string(contents)


'''
----------------
EXTRACTION
----------------
'''
#for test, we'll be querying

start=pd.Timestamp('202101010000', tz='Europe/Berlin')
end=pd.Timestamp('202101010600', tz='Europe/Berlin')
country_code= 'DE_TENNET'
country_code_from=''
country_code_to=''
type_marketagreement_type=''
contract_marketagreement_type=''

start2=pd.Timestamp('20210101', tz='Europe/Berlin')
end2=pd.Timestamp('20211231', tz='Europe/Berlin')

pd.set_option('display.max_rows', None)

try:
    entsoe_data = entsoe_client.query_generation(country_code, start=start, end=end)
    entsoe_dict = xmltodict.parse(entsoe_data)
    entsoe_json = json.dumps(entsoe_dict, indent=4, ensure_ascii=False)
except Exception as e:
    print("An exception occurred:", e)


'''
----------------
LOAD
----------------
'''
#upload to GCS
landing_filename=f"entsoe_data_{country_code}.json"
upload_blob_to_gcs(bucket_name=gcs_bucket, contents=entsoe_json, destination_blob_name=landing_filename)



23/01/03 08:00:51 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 29768581 ms exceeds timeout 120000 ms
23/01/03 08:00:51 WARN SparkContext: Killing executors is not supported by current scheduler.


In [16]:
## COBA SPARK GCS CONNECTOR ##


# berangkat pak haji
# setup parameternya
gcs_bucket = gcs_bucket
path = f"gs://{gcs_bucket}/{landing_filename}"
# path ="/home/rafzul/projects/entsoe-pipelines/sample.xml"

#coba spark gcs connector
#setup sparksession for entry point - COBA GCS CONNECTOR
SPARK_HOME = os.environ["SPARK_HOME"]
spark = SparkSession.builder.appName("gcp_playground") \
    .config("spark.jars", f"{SPARK_HOME}/jars/gcs-connector-hadoop3-latest.jar, {SPARK_HOME}/jars/spark-bigquery-with-dependencies_2.13-0.27.1.jar") \
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "google.cloud.hadoop.fs.gcs.GoogleHadoopFS") \
    .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", service_account_file) \
    .getOrCreate()



In [52]:
# #create schema to be enforced in subsequent json load operation
# with open("schema_raw.json", "w") as schrawjson:
#     schrawjson.write(df_spark_orig.schema.json())

In [17]:
with open("schema_raw2.json", "r") as schrawjson:
    json_schema_data = schrawjson.read()
    json_enforced_schema = StructType.fromJson(json.loads(json_schema_data))
    

In [18]:
#create dataframe from gcs
path = f"gs://{gcs_bucket}/{landing_filename}"
print(path)
df_spark = spark.read.format("json").schema(json_enforced_schema) \
   .option("header","true") \
   .option("multiLine","true") \
   .load(path) \
   .select("GL_MarketDocument.*")
   

gs://entsoe_analytics_1009/entsoe_data_DE_TENNET.json


In [9]:
#clean non timeseries column name. change dot to underscores
df_spark = df_spark.toDF(*(c_name.replace(".", "_") for c_name in df_spark.columns))

#clean timeseries column. cast TimeSeries to new scheme where 1.dot in names are replaced with underscores and 2. Strange characters such as '@' or '#' are removed
ts_schema = df_spark.select("TimeSeries").dtypes[0][1]
replacements = [('\.', '_'), ('[@#]', '')]
for old, new in replacements:
    ts_schema = re.sub(old, new, ts_schema)
df_spark = df_spark.withColumn("TimeSeries", (F.col("TimeSeries").cast(ts_schema)))

In [10]:
#flatten column

# explode timeseries column into struct
df_spark = df_spark.withColumn("TimeSeries", F.explode("TimeSeries"))


In [11]:
df_spark.printSchema()

root
 |-- @xmlns: string (nullable = true)
 |-- mRID: string (nullable = true)
 |-- revisionNumber: string (nullable = true)
 |-- type: string (nullable = true)
 |-- process_processType: string (nullable = true)
 |-- sender_MarketParticipant_mRID: struct (nullable = true)
 |    |-- #text: string (nullable = true)
 |    |-- @codingScheme: string (nullable = true)
 |-- sender_MarketParticipant_marketRole_type: string (nullable = true)
 |-- receiver_MarketParticipant_mRID: struct (nullable = true)
 |    |-- #text: string (nullable = true)
 |    |-- @codingScheme: string (nullable = true)
 |-- receiver_MarketParticipant_marketRole_type: string (nullable = true)
 |-- createdDateTime: string (nullable = true)
 |-- time_Period_timeInterval: struct (nullable = true)
 |    |-- end: string (nullable = true)
 |    |-- start: string (nullable = true)
 |-- TimeSeries: struct (nullable = true)
 |    |-- MktPSRType: struct (nullable = true)
 |    |    |-- psrType: string (nullable = true)
 |    |-- P

In [97]:


# flatten TimeSeries if there is TimeSeries, flatten AttributeInstanceComponent if there is AttributeInstanceComponent
def unpack_df(nested_df):
    component_columns = ["TimeSeries", "AttributeInstanceComponent"]
    general_cols = [c for c in nested_df.columns if c not in component_columns]
    if "TimeSeries" in nested_df.columns:
        data_cols_name = "TimeSeries"
        data_cols = [c for c in nested_df.select("TimeSeries.*").columns]
    else:
        pass
    unpacked_df = nested_df.select(general_cols \
                                   + [F.col(data_cols_name+"."+c).alias(data_cols_name+"_"+c)\
                                      for c in data_cols])
    return unpacked_df
    

df_spark = unpack_df(df_spark)
df_spark.printSchema()

root
 |-- @xmlns: string (nullable = true)
 |-- mRID: string (nullable = true)
 |-- revisionNumber: string (nullable = true)
 |-- type: string (nullable = true)
 |-- process_processType: string (nullable = true)
 |-- sender_MarketParticipant_mRID: struct (nullable = true)
 |    |-- #text: string (nullable = true)
 |    |-- @codingScheme: string (nullable = true)
 |-- sender_MarketParticipant_marketRole_type: string (nullable = true)
 |-- receiver_MarketParticipant_mRID: struct (nullable = true)
 |    |-- #text: string (nullable = true)
 |    |-- @codingScheme: string (nullable = true)
 |-- receiver_MarketParticipant_marketRole_type: string (nullable = true)
 |-- createdDateTime: string (nullable = true)
 |-- time_Period_timeInterval: struct (nullable = true)
 |    |-- end: string (nullable = true)
 |    |-- start: string (nullable = true)
 |-- TimeSeries_MktPSRType: struct (nullable = true)
 |    |-- psrType: string (nullable = true)
 |-- TimeSeries_Period: struct (nullable = true)
 | 

In [102]:
#tulis data ke bigquery via temporary gcs bucket
df_spark.write \
  .format("bigquery") \
  .option("project","rafzul-analytics-1009") \
  .option("temporaryGcsBucket","entsoe_temp_1009") \
  .mode("append") \
  .save("rafzul-analytics-1009.entsoe_playground.fact_test")

In [ ]:
# if df_spark2.select("TimeSeries"):
#     # rename all column (specifically replace dot with underscore) on every struct nested in TimeSeries array column. use transform
#     df_spark3 = df_spark3.withColumn("TimeSeries", F.transform \
#     ("TimeSeries", lambda el,ind: \
#     F.struct \
#     ("abc" for c_name in el.columns)))
    
# # # #get list of names of all columns in every struct inside TimeSeries array column
# # a = [x for x in df_spark2.select(("TimeSeries"))]
# # print(a)

# df_spark3.select("TimeSeries").show(10, truncate=False)
# a = F.struct(F.col("TimeSeries").getItem(c_name).alias(c_name.replace(".", "_")) for i, c_name in enumerate(df_spark2.schema["TimeSeries"].dataType.elementType.fieldNames()))
# # print(a


# F.struct(F.col("TimeSeries") for c_name in df_spark2.schema[x].dataType.names


# df_spark2.show()
# df_spark2.printSchema()

# # #get list of names of all columns in every struct inside TimeSeries array column
# a = [(c_name,i) for (c_name,i) in enumerate(df_spark2.schema["TimeSeries"].dataType.elementType.fieldNames())]
# print(a)


# df_spark2 = df_spark2.withColumn("TimeSeries", F.transform("TimeSeries", lambda x: F.struct(*[F.col("TimeSeries." + c_name.replace(".", "_")).alias(c_name) for c_name in x])))

In [16]:
# #for non timeseries column
# df_spark2 = df_spark2.toDF(*(c_name.replace(".", "_") for c_name in df_spark2.columns))
# b = df_spark2.select(F.col("TimeSeries"))
# #for timeseries one
# a = F.struct(F.col("TimeSeries").getItem(c_name).alias(c_name.replace(".", "_")) for i, c_name in enumerate(df_spark2.schema["TimeSeries"].dataType.elementType.fieldNames()))
# print(a)
# if df_spark2.select("TimeSeries"):
#     # rename all column (specifically replace dot with underscore) on every struct nested in TimeSeries array column. use transform, use Timeseries schema and its datatype.name o
#     df_spark2 = df_spark2.withColumn("TimeSeries", F.transform("TimeSeries", lambda x: (a)))
    
# # #get list of names of all columns in every struct inside TimeSeries array column
# # a = [c_name for c_name in enumerate(df_spark2.schema["TimeSeries"].dataType.elementType.fieldNames())]


# df_spark3.show()
# df_spark3.printSchema()



# # df_spark2 = df_spark2.withColumn("TimeSeries", F.transform("TimeSeries", lambda x: F.struct(*[F.col("TimeSeries." + c_name.replace(".", "_")).alias(c_name) for c_name in x])))

In [134]:
# #cast schema with the dot replaced with underscore, programatically


# ---- solution for  column names beside TimeSeries:
# changed_column_general = [(column_name, column_name.replace(".", "_")) for column_name in df_schema.fieldNames() if "." in x]
# for column_name, changed_column_name in changed_column_general:
#     df_schema = df_spark2.select([F.col(c).alias(mappingcolumn_name, changed_column_name)
# # UPDATE: there is even better solution

# #changed non timeseries column
# df_spark2 = df_spark2.toDF(*(c.replace(".", "_") for c in df_spark2.columns))

In [76]:
with open("example_destination.json", "w+") as output_file:
    output_file.write(df_spark2.toJSON())

TypeError: write() argument must be str, not RDD

In [ ]:
# #inferring schema and get the data type of each column and turn it into spark dataframe
# datatype_infer = pd.DataFrame.from_dict(xml_file_dict[0], orient='index')

In [ ]:
# #flatten nested df at every layer
# from pyspark.sql.types import *
# from pyspark.sql.import functions as f

# def flatten_structs(nested_df):
#     stack = [(), nested_df]
#     columns = []
    
#     while len(stack) > 0:
#         parents
        
    